# Overview
An enviroment to train and evaluate neural networks on rule learning. The language used is stated in the notions of Chomsky.
- Terminal symbols (encoded as integers)
- Nonterminalsymbols and production rules (encoded as lambda-functions)

This is how it works:
1. Section "Libraries and utilities imports libraries and defines useful functions.
2. Section "Data generation" defines functions and a class for generating the training data
3. Section "Networks" defines a function that implements different network architectures.
4. Uses the before defined functions and Derivation class.
  a. "get_training_data()" produces data using the grammar and logic for building derivations in the Derivation class the nonterminals and production rules used by "nonterms()".
  b. A loop using "evaluate()" trains all specified architectures on the data and plots the results.

In [13]:
from libs import *

In [20]:
importlib.reload(util)

<module 'util' from '/home/str/university/masters thesis/master-s-thesis/util.py'>

In [4]:
inpt, outp, outp_dict = gnrn.dat_b(itra = [1,3], s_si = 2000)
inpt_flat = []
for i in inpt:
    inpt_flat.append(util.lflt(i))
inpt = util.pad(inpt_flat)
inpt_ten = []
for i in inpt:
    iten = torch.tensor(i)
    inpt_ten.append(iten)
inpt = []
padding = []
for i in list(range(t_nu+7)):
    padding.append(0)
padding = torch.tensor(padding)
for i in inpt_ten:
    ione = F.one_hot(i[1:], num_classes=t_nu+8)
    index = torch.cat((i[0].unsqueeze(0), padding))
    conc = torch.cat((index.unsqueeze(0), ione), dim=0)
    inpt.append(conc)
inpt = np.array(inpt)
inpt = torch.tensor(inpt, dtype= torch.float32)
inpt = inpt.view(inpt.size(0),-1)
inpt = torch.cat((inpt[:,:1],inpt[:,13:]), dim = 1)
from tqdm import tqdm
# Print results
print(f"Input: {inpt}")
print(f"Output: {outp[0]}")
print(f"Number inputs: {len(inpt)}")
print(f"Number outputs: {len(outp)}")
print(f"Average comparing examples per input: {len(outp)/len(inpt)}")

Padded exmaples from X: 100%|██████████| 2000/2000 [00:01<00:00, 1592.03it/s]


Input: tensor([[0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.9970e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.9980e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.9990e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])
Output: [1, 4, 1, [1, 11, 5], [1, 10, 1]]
Number inputs: 2000
Number outputs: 2216
Average comparing examples per input: 1.108


In [6]:
outp_flt = []
for i in outp:
    outp_flt.append(util.lflt(i))

In [23]:
#dataloader
dataloader = DataLoader(dataset=inpt,shuffle=True,batch_size=50)

In [24]:
outp_max_length = len(max(outp_flt, key = len)) * 13
model = arch.net(inpt.shape[1]-1,outp_max_length)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
epochs = 100

In [18]:
# Template from GPT-4, going to be edited:
def s_ls(y_pred, x_train, outp, outp_dict):
    counter = 0
    loss_batch_list = []
    while counter < len(y_pred):
        val_i = y_pred[counter]
        x_i = x_train[counter] 
        y_train = outp_dict[int(x_i[0])]
        if len(y_train) > 0:
            y_train = y_train[0]
        y_train = util.convert_single(y_train, outp_flt)
        loss_batch_list.append(torch.mean((val_i - y_train) ** 2))
        counter += 1
#    loss = torch.mean(loss_batch_list)
    #return loss
    return loss

In [25]:
costval = []

for j in range(epochs):
    for i, x_train in enumerate(dataloader):
        #prediction
        y_pred = model(x_train[:,1:])
        cost = s_ls(y_pred, x_train, outp, outp_dict)
        # Backpropagation
        optimizer.zero_grad()
        cost.backward()
        optimizer.step(print)

    if j % 50 == 0:
        print(cost)
        costval.append(cost.item())  # Use .item() to store the loss value as a number









































tensor(0.1461, grad_fn=<MeanBackward0>)
























































































































































































































































































































































































































































































































































































































































































































































































































































































































































